In [23]:
import re
import string
import pandas as pd
from math import log
import os
import random
import re
import string
from collections import Counter
from sklearn.cluster import KMeans

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

from transformers import AutoTokenizer, AutoModel
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch
import hnswlib

[nltk_data] Error loading stopwords: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>


In [2]:
final_counts = []

In [3]:
final_counts = []
def clean_text(text):
    text = str(text).lower().strip() 
    text = re.sub("(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", text)
    text = re.sub('[^a-z\s]+', "", text)
    # text = text.translate (str.maketrans('', '', string.punctuation))
      # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    # text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [7]:
all_roman_words = dict() 
pref = "/Volumes/olaaaa/Thesis_backup/data/"
for file in os.listdir("/Volumes/olaaaa/Thesis_backup/data"):
    #print(file, "here")
    if file.endswith("csv"):
        # print("here")
        df = pd.read_csv(pref+file, header = None, low_memory=False)
        # print(len(df))
        df = df[[0,21]]
        df.columns = ['reg_no', 'subject_content']
        
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        for sent in df.subject_content_cleaned.values:
            res = set(re.findall(r'\w+', sent))
            for word in res:
                try:
                    all_roman_words[word] += 1
                except:
                    all_roman_words[word] = 1
                
            # print(all_roman_words)
        # tmp = [ val for val in df['subject_content_cleaned'].values \
        # if len(re.findall("[a-z]+",val)) > 0.5*len(val.split())]
        # all_roman.extend(tmp) 
        print(file, "Size of updated set", str(len(all_roman_words)))
        # print()
        break
        
        
        

chunk8.csv Size of updated set 777206


In [10]:
hindi_latin = set()
all_words = set()
with open("word.list") as f:
    for line in f.readlines():
        line = line.strip()
        all_words.add(line)

In [14]:
for key in all_roman_words.keys():
    # print(key)
    if key not in all_words:
        hindi_latin.add(key)

In [16]:
hindi_latin

{'rukvavo',
 'morningif',
 'dhaneshkumar',
 'uselesswe',
 'verdan',
 'toocomplaints',
 'rupaiyah',
 'conist',
 'budyut',
 'majura',
 'aramy',
 'tooardaalkgrupaye',
 'crcr',
 'transfersfrom',
 'gulshn',
 'mohanjit',
 'logoky',
 'adalton',
 'matchesquot',
 'ssecicmckyq',
 'comfortablity',
 'deliking',
 'govemment',
 'kalainaveenmohan',
 'isla',
 'theyineligible',
 'mahimmumbai',
 'aiese',
 'facilitiesf',
 'muhaale',
 'nudesex',
 'adhikaaryo',
 'bahana',
 'icuward',
 'suverying',
 'passengerpublic',
 'sangasthan',
 'dwar',
 'rewai',
 'harmfulchemical',
 'barneka',
 'factorygodhaun',
 'saktimeri',
 'introdue',
 'handsanatizer',
 'bhavyesh',
 'rejalte',
 'disconnet',
 'raisai',
 'rummyother',
 'sambhartu',
 'doktro',
 'examhoge',
 'koon',
 'fuchter',
 'intelleigencecoimbatoretamilnadu',
 'quotquotrquotquot',
 'dbfn',
 'registrymappayment',
 'publicationsindia',
 'seetaraman',
 'broadcastil',
 'topmoindia',
 'officialsadministratorshospitals',
 'hmn',
 'commrcexchn',
 'tapandubey',
 'monthsw

In [17]:
def freq(sent):
    arr = np.zeros(26)
    for letter in sent:
        if 97 <= ord(letter) <= 122:
            # print("here")
            arr[ord(letter) - 97] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [18]:
def fit_hnsw_index(features, ef=100, M=16):
    num_elements = len(features)
    labels_index = np.arange(num_elements)
    EMBEDDING_SIZE = len(features[0])
    
    p = hnswlib.Index(space='l2', dim=EMBEDDING_SIZE)# possible space options are l2, cosine or ip
    p.init_index(max_elements=num_elements, ef_construction=ef, M=M)
    int_labels = p.add_items(features, labels_index)
    p.set_ef(ef) 
    return p

In [19]:
df = pd.DataFrame(hindi_latin, columns= ['word'])
df['histograms'] = df['word'].apply(freq)
df.reset_index(drop=True, inplace=True)

In [20]:
df

,word,histograms
0,rukvavo,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,morningif,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.2773500981126146, ..."
2,dhaneshkumar,"[0.5, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.5, 0.0..."
3,uselesswe,"[0.0, 0.0, 0.0, 0.0, 0.6546536707079772, 0.0, ..."
4,verdan,"[0.4082482904638631, 0.0, 0.0, 0.4082482904638..."
...,...,...
709903,careerlivelihood,"[0.18257418583505536, 0.0, 0.18257418583505536..."
709904,puzhavoram,"[0.5773502691896258, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
709905,kapoorsat,"[0.5547001962252291, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
709906,chdillip,"[0.0, 0.0, 0.2886751345948129, 0.2886751345948..."


In [21]:
K = 20 #nearest neighbours you wish to find for each grievance
l2_param = 0.001 #threshold for l2 distance
min_docs = 5 #minimum number of grievances in a set to call it a bulk

In [24]:
X = [item.tolist() for item in df['histograms'].values]

p = fit_hnsw_index(X, ef=K*10)
ann_neighbor_indices, ann_distances = p.knn_query(X, K)

In [25]:
ann_neighbor_indices

array([[     0, 207665, 635950, ..., 508717, 179981, 533456],
       [     1, 405084, 219654, ..., 152946, 189539, 272519],
       [     2, 138149, 707381, ..., 416024, 477741, 307206],
       ...,
       [709905, 221761, 573834, ..., 463120, 585344, 675751],
       [709906, 359525, 158740, ..., 441406, 560756, 676981],
       [709907, 515812, 187273, ..., 353993,   9145, 226985]],
      dtype=uint64)

In [26]:
primary_dict = {} # dictionary of documentid versus a dictionary of neighbours and their corresponding distances

for i in range(len(ann_neighbor_indices)):
    indices = ann_neighbor_indices[i]
    distances = ann_distances[i]
    tmp = {}
    for k,v in zip(indices, distances):
        tmp[k] = v
    primary_dict[i] = tmp

# bulk_set = set()
bulk_all = dict()

# i = 1
for id in primary_dict.keys():
    tmp_docs = []
    # if id in bulk_set:
    #     continue
    tmp_docs.append(df.loc[id, "word"])
    # print(id)
    for k,v in primary_dict[id].items():
        # print(k,v)
        if k == id: 
            continue
        if float(v) < l2_param:
            tmp_docs.append(df.loc[k, "word"])
            # bulk_set.add(k)

    # print(tmp_docs)
    if len(tmp_docs)>min_docs:
        bulk_all[df.loc[id, "word"]] = tmp_docs
        # i+=1
bulk_all

{'verdan': ['verdan',
  'vendra',
  'devran',
  'nardve',
  'vandre',
  'randev',
  'vendar',
  'devnar',
  'vedran',
  'vander',
  'nardev'],
 'conist': ['conist',
  'sction',
  'ctions',
  'tiscon',
  'consti',
  'tcsion',
  'notics'],
 'majura': ['majura',
  'umaraj',
  'ramauj',
  'majaur',
  'jamuar',
  'marjau',
  'jamaur'],
 'aramy': ['aramy',
  'ramya',
  'amray',
  'ramay',
  'rayam',
  'maary',
  'maray',
  'aarmy',
  'aryma',
  'mayar',
  'mayra',
  'myraa',
  'marya',
  'amary',
  'armay'],
 'crcr': ['crcr',
  'crrc',
  'rc',
  'rcrc',
  'cr',
  'crcrcrcrcrcrcrcrcrcrcrcrcrcrcrcrcrcr'],
 'isla': ['isla',
  'isal',
  'alis',
  'lais',
  'silsilaa',
  'sali',
  'lias',
  'ilsa',
  'alsi',
  'lisa',
  'aisl',
  'asli',
  'lasi'],
 'aiese': ['aiese', 'eaise', 'easie', 'isaee', 'aisee', 'aeise'],
 'dwar': ['dwar',
  'wardward',
  'wdra',
  'rdwa',
  'rawd',
  'drwa',
  'awrd',
  'dwra',
  'wrad',
  'wadr'],
 'barneka': ['barneka',
  'banaker',
  'erbakan',
  'karbane',
  'bakaner

In [29]:
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

for key in bulk_all.keys():
    bulk_all[key] = sorted(bulk_all[key], key= lambda x: edit_distance(x, key))[:10]



In [31]:
def count_total_hashes(sent, tokenizer):
    tot = 0
    lst  = tokenizer.tokenize(sent)
    # print(lst)
    for word in lst:
        tot += word.count('#')
    return tot

tokenizer_salesken = AutoTokenizer.from_pretrained("salesken/similarity-eng-hin_latin")

def find_word_with_minimum_hashes(lst):
    lst = sorted(lst, key = lambda x: count_total_hashes(x, tokenizer_salesken))
    # print(lst)
    return lst[0]

final_wording = dict()

for key in bulk_all.keys():
    val = bulk_all[key]
    word = find_word_with_minimum_hashes(val)
    for v in val:
        final_wording[v] = word


In [34]:
# ['karan', 'nkara', 'krana', 'narak', 'karna', 'krnaa']

In [35]:
print(bulk_all['karan'])
print(bulk_all['karna'])


['karan', 'narak', 'karna', 'krana', 'nkara', 'kanra', 'akran', 'kaarn', 'narka', 'nakar']
['karna', 'narka', 'karan', 'krnaa', 'rakna', 'krana', 'nkara', 'kanra', 'kaarn', 'ranka']


In [36]:
edit_distance("karna", "rakna")

2

In [37]:
find_word_with_minimum_hashes(['karan', 'nkara', 'krana', 'narak', 'karna'])

'karan'

In [38]:
find_word_with_minimum_hashes(['karna', 'karan', 'nkara', 'krana', 'krnaa'])

'karna'

In [39]:
final_wording = dict()

for key in bulk_all.keys():
    val = bulk_all[key]
    word = find_word_with_minimum_hashes(val)
    for v in val:
        final_wording[v] = word

In [41]:
final_wording

{'verdan': 'vendar',
 'vendar': 'vendar',
 'vedran': 'vendar',
 'vendra': 'vendar',
 'devran': 'vendar',
 'nardve': 'vendar',
 'vandre': 'vendar',
 'devnar': 'vendar',
 'vander': 'vendar',
 'nardev': 'vendar',
 'conist': 'notics',
 'consti': 'notics',
 'ctions': 'notics',
 'notics': 'notics',
 'sction': 'notics',
 'tcsion': 'notics',
 'tiscon': 'notics',
 'majura': 'jamaur',
 'majaur': 'jamaur',
 'umaraj': 'jamaur',
 'jamuar': 'jamaur',
 'marjau': 'jamaur',
 'jamaur': 'jamaur',
 'ramauj': 'jamaur',
 'aramy': 'amray',
 'ramya': 'amray',
 'amray': 'amray',
 'ramay': 'amray',
 'maray': 'amray',
 'aarmy': 'amray',
 'aryma': 'amray',
 'amary': 'amray',
 'armay': 'amray',
 'rayam': 'maray',
 'crcr': 'rc',
 'crrc': 'rc',
 'rc': 'rc',
 'rcrc': 'rc',
 'cr': 'rc',
 'crcrcrcrcrcrcrcrcrcrcrcrcrcrcrcrcrcr': 'rc',
 'isla': 'asli',
 'isal': 'asli',
 'ilsa': 'lasi',
 'lisa': 'lasi',
 'aisl': 'lasi',
 'asli': 'lasi',
 'sali': 'lasi',
 'alis': 'lasi',
 'lais': 'lasi',
 'silsilaa': 'isla',
 'aiese': 'aei

In [42]:
len(bulk_all)

63785

## To Do
1. We need to try trie/tree approach (auto-correct). Read about it
2. Check the translation time complexity using m2m for gujarati and tamil
